In [1]:
import tensorflow as tf


In [2]:
epoch = 15
batch_size = 100

In [3]:
xdata = [[[1,0,0,0], # 4*4  #h
        [0,1,0,0],#e
        [0,0,1,0],#l
        [0,0,0,1]]]#o

ydata = [[0,1,0,0], # 5*4
        [0,0,1,0],
        [0,0,1,0],
        [0,0,0,1]]

X = tf.placeholder(tf.float32, [None,4,4])
Y = tf.placeholder(tf.float32, [None, 4])

W = tf.Variable(tf.random_normal([4, 1]))
b = tf.Variable(tf.random_normal([1]))

In [4]:
cell = tf.nn.rnn_cell.LSTMCell(4)
cell.state_size

LSTMStateTuple(c=4, h=4)

In [5]:
outputs, states = tf.nn.dynamic_rnn(cell, X, dtype=tf.float32)

In [6]:
#outputs = tf.transpose(outputs, [1, 0, 2])
outputs = outputs[-1]
logits = tf.matmul(outputs, W) + b
H = tf.nn.softmax(logits)

In [7]:
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=Y))
train = tf.train.AdamOptimizer(0.0001).minimize(cost)

In [8]:
is_correct = tf.equal(tf.argmax(logits, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))

In [9]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [10]:

for e in range(epoch):
    total_cost = 0
    print(e+1)
    for i in range(10):
        sess.run([train],feed_dict={X: xdata, Y: ydata})
        c = sess.run(cost, feed_dict={X: xdata, Y: ydata})
        print(c)
    #    total_cost += _cost
   # a = sess.run(accuracy, feed_dict={X: mnist.test.images.reshape(test_batch_size, 28, 28), Y:mnist.test.labels})
  #  print(a)


1
1.3862944
1.3862944
1.3862944
1.3862944
1.3862944
1.3862944
1.3862944
1.3862944
1.3862944
1.3862944
2
1.3862944
1.3862944
1.3862944
1.3862944
1.3862944
1.3862944
1.3862944
1.3862944
1.3862944
1.3862944
3
1.3862944
1.3862944
1.3862944
1.3862944
1.3862944
1.3862944
1.3862944
1.3862944
1.3862944
1.3862944
4
1.3862944
1.3862944
1.3862944
1.3862944
1.3862944
1.3862944
1.3862944
1.3862944
1.3862944
1.3862944
5
1.3862944
1.3862944
1.3862944
1.3862944
1.3862944
1.3862944
1.3862944
1.3862944
1.3862944
1.3862944
6
1.3862944
1.3862944
1.3862944
1.3862944
1.3862944
1.3862944
1.3862944
1.3862944
1.3862944
1.3862944
7
1.3862944
1.3862944
1.3862944
1.3862944
1.3862944
1.3862944
1.3862944
1.3862944
1.3862944
1.3862944
8
1.3862944
1.3862944
1.3862944
1.3862944
1.3862944
1.3862944
1.3862944
1.3862944
1.3862944
1.3862944
9
1.3862944
1.3862944
1.3862944
1.3862944
1.3862944
1.3862944
1.3862944
1.3862944
1.3862944
1.3862944
10
1.3862944
1.3862944
1.3862944
1.3862944
1.3862944
1.3862944
1.3862944
1.3862944

In [12]:
sess.run(logits, feed_dict={X:[[[1,0,0,0], # 4*4  #h
        [0,1,0,0],#e
        [0,0,1,0],#l
        [0,0,0,1]]]})

array([[-0.04132923],
       [-0.01540568],
       [ 0.0065751 ],
       [-0.11918122]], dtype=float32)

In [ ]:
cell = tf.nn.rnn_cell.LSTMCell(100)
outputs, states = tf.nn.dynamic_rnn(cell, X, dtype=tf.float32)
outputs = tf.transpose(outputs, [1, 0, 2])
outputs = outputs[-1]
logits = tf.matmul(outputs, W) + b

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=Y))
train = tf.train.AdamOptimizer(0.0001).minimize(cost)

is_correct = tf.equal(tf.argmax(logits, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))

sess = tf.Session()
sess.run(tf.global_variables_initializer())

total_batch = int(mnist.train.num_examples/batch_size)
for e in range(epoch):
    total_cost = 0
    print(e+1)
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        batch_xs = batch_xs.reshape((batch_size, 28, 28))

        _, _cost = sess.run([train, cost],feed_dict={X: batch_xs, Y: batch_ys})
        total_cost += _cost
    test_batch_size = len(mnist.test.images)
    a = sess.run(accuracy, feed_dict={X: mnist.test.images.reshape(test_batch_size, 28, 28), Y:mnist.test.labels})
    print(a)


In [11]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.python.ops.rnn_cell_impl import _zero_state_tensors

class CapRNNcell(tf.contrib.rnn.RNNCell):
    def __init__(self, input_dim):
        self.input_dim = input_dim

        self.W = tf.get_variable("W", [self.input_dim , 1], tf.float32)
        self.b = tf.get_variable("b", [1])

    @property
    def state_size(self):
        return 1

    @property
    def output_size(self):
        return 1

    def __call__(self, inputs, state):
        output =state*tf.nn.sigmoid(tf.matmul(inputs, self.W)+ self.b)

        return output, output

def CapRnnModel(timeSeries_before_forgetting_gate, init_cap):

    cap_cell = CapRNNcell(input_dim=3)
    cap_series, final_cap = tf.nn.dynamic_rnn(cell=cap_cell, inputs=timeSeries_before_forgetting_gate, initial_state=init_cap)

    return  cap_series , final_cap

x_place=tf.placeholder(tf.float32 , [1,2,3])
init_cap_place=tf.placeholder(tf.float32 , [1,1])

y=CapRnnModel(x_place, init_cap_place)

with tf.Session() as sess:
    sess.run(tf.initialize_all_variables())
    a=np.random.rand(1,2,3)
    b=np.random.rand(1,1)
    result=sess.run(y,feed_dict={x_place:a , init_cap_place:b})
    print(result)

Instructions for updating:
Use `tf.global_variables_initializer` instead.
(array([[[0.04323792],
        [0.04104751]]], dtype=float32), array([[0.04104751]], dtype=float32))


In [ ]:
sess.run([train],feed_dict={X: xdata, Y: ydata})